# Common imports & library functions

In [3]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
import math
import re

# Day 6: Custom Customs

In [84]:
_sanitize_re = re.compile('[^a-z]')
def sanitize(responses):
    """
    >>> sanitize('ab\\n c d\\ng')
    'abcdg'
    >>> sanitize('abcxyz123')
    'abcxyz'
    """
    return _sanitize_re.sub('', responses)

_union = Counter
_intersection = lambda sets: set.intersection(*sets)
_num_anyone = lambda r: len(_union(sanitize(r)))
_num_everyone = lambda r: len(_intersection(set(sanitize(a)) for a in r.split('\n')))

def num_yes_questions(responses, count_method):
    return sum(count_method(r.strip()) for r in responses.split('\n\n'))

def num_anyone_yes_questions(responses):
    """
    >>> num_anyone_yes_questions('abcx\\nabcy\\nabcz')
    6
    >>> num_anyone_yes_questions('''
    ...     abc \\n
    ...     a \\n b \\n c \\n
    ...     ab \\n ac \\n
    ...     a \\n a \\n a \\n a \\n
    ...     b
    ... ''')
    11
    """
    return num_yes_questions(responses, _num_anyone)

def num_everyone_yes_questions(responses):
    """
    >>> num_everyone_yes_questions('abcx\\nabcy\\nabcz')
    3
    >>> num_everyone_yes_questions('''
    ...     abc \\n
    ...     a \\n b \\n c \\n''')
    3
    >>> num_everyone_yes_questions('''
    ...     abc \\n
    ...     a \\n b \\n c \\n
    ...     ab \\n ac \\n
    ...     a \\n a \\n a \\n a \\n
    ...     b
    ... ''')
    6
    """
    return num_yes_questions(responses, _num_everyone)

In [85]:
doctest.run_docstring_examples(sanitize, globs=None, verbose=True)
doctest.run_docstring_examples(num_anyone_yes_questions, globs=None, verbose=True)
doctest.run_docstring_examples(num_everyone_yes_questions, globs=None, verbose=True)

Finding tests in NoName
Trying:
    sanitize('ab\n c d\ng')
Expecting:
    'abcdg'
ok
Trying:
    sanitize('abcxyz123')
Expecting:
    'abcxyz'
ok
Finding tests in NoName
Trying:
    num_anyone_yes_questions('abcx\nabcy\nabcz')
Expecting:
    6
ok
Trying:
    num_anyone_yes_questions('''
        abc \n
        a \n b \n c \n
        ab \n ac \n
        a \n a \n a \n a \n
        b
    ''')
Expecting:
    11
ok
Finding tests in NoName
Trying:
    num_everyone_yes_questions('abcx\nabcy\nabcz')
Expecting:
    3
ok
Trying:
    num_everyone_yes_questions('''
        abc \n
        a \n b \n c \n''')
Expecting:
    3
ok
Trying:
    num_everyone_yes_questions('''
        abc \n
        a \n b \n c \n
        ab \n ac \n
        a \n a \n a \n a \n
        b
    ''')
Expecting:
    6
ok


In [86]:
# Final answers
with open('day6.txt') as f:
    responses = f.read().strip()
    print('Part 1: ', num_anyone_yes_questions(responses))
    print('Part 1: ', num_everyone_yes_questions(responses))

Part 1:  6387
Part 1:  3039


# Day 7: Handy Haversacks

In [120]:
def parse_rule(rule):
    """
    >>> parse_rule('1 bright white bag')
    (1, 'bright white')
    >>> parse_rule('2 yellow bags')
    (2, 'yellow')
    """
    parts = rule.split()
    return int(parts[0]), ' '.join(parts[1:-1])

def parse_rules(rules):
    """
    >>> parse_rules('5 faded blue bags, 6 dotted black bags')
    [(5, 'faded blue'), (6, 'dotted black')]
    >>> parse_rules('no other bags')
    []
    """
    if rules == 'no other bags': return []
    return [parse_rule(rule.strip()) for rule in rules.split(',')]

def parse_regulations(regulations):
    mapping = {}
    regulations = regulations.split('.')
    for regulation in regulations:
        regulation = regulation.strip()
        if not regulation: continue
        color, rules = regulation.strip().split('bags contain')
        mapping[color.strip()] = parse_rules(rules.strip()) 
    return mapping

def count_ancestors(color, regulations):
    parent_links = defaultdict(set)
    for parent, children in regulations.items():
        for _, child in children:
            parent_links[child].add(parent)
    
    ancestors = set()
    to_visit = {color}
    while to_visit:
        child = to_visit.pop()
        parents = parent_links[child]
        to_visit |= parents
        ancestors |= parents 
    return len(ancestors)

from functools import lru_cache

def bags_contained(color, regulations):
    @lru_cache(len(regulations))
    def _bags_contained(color):
        children = regulations[color]
        if not children: return 0
        return sum(count * (_bags_contained(child) + 1)
                   for count, child in children)
    return _bags_contained(color)


In [63]:
doctest.run_docstring_examples(parse_rule, globs=None, verbose=True)
doctest.run_docstring_examples(parse_rules, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_rule('1 bright white bag')
Expecting:
    (1, 'bright white')
ok
Trying:
    parse_rule('2 yellow bags')
Expecting:
    (2, 'yellow')
ok
Finding tests in NoName
Trying:
    parse_rules('5 faded blue bags, 6 dotted black bags')
Expecting:
    [(5, 'faded blue'), (6, 'dotted black')]
ok
Trying:
    parse_rules('no other bags')
Expecting:
    []
ok


In [121]:
test_regulations = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""
expected_mapping = {
    'light red': [(1, 'bright white'), (2, 'muted yellow')],
    'dark orange': [(3, 'bright white'), (4, 'muted yellow')],
    'bright white': [(1, 'shiny gold')],
    'muted yellow': [(2, 'shiny gold'), (9, 'faded blue')],
    'shiny gold': [(1, 'dark olive'), (2, 'vibrant plum')],
    'dark olive': [(3, 'faded blue'), (4, 'dotted black')],
    'vibrant plum': [(5, 'faded blue'), (6, 'dotted black')],
    'faded blue': [],
    'dotted black': []
}
assert parse_regulations(test_regulations) == expected_mapping
assert count_ancestors('shiny gold', expected_mapping) == 4

In [122]:
test_regulations = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
"""
assert bags_contained('shiny gold', parse_regulations(test_regulations)) == 126

In [123]:
# Final answers
with open('day7.txt') as f:
    regulations = parse_regulations(f.read().strip())
    print('Part 1: ', count_ancestors('shiny gold', regulations))
    print('Part 2: ', bags_contained('shiny gold', regulations))

Part 1:  103
Part 2:  1469


# Day 8: Handheld Halting

In [53]:
from enum import auto, Enum

def parse_instruction(instr):
    """
    >>> parse_instruction('acc -99')
    ('acc', -99)
    >>> parse_instruction('nop +0')
    ('nop', 0)
    """
    op, arg = instr.split()
    return op, int(arg)

def parse_instructions(instrs):
    return [parse_instruction(i)
            for i in instrs.strip().splitlines()]

class State(Enum):
    TERMINATED = auto()
    RUNNING = auto()
    LOOPED = auto()

    def __repr__(self):
        return '<%s.%s>' % (self.__class__.__name__, self.name)

def execute_until_loop(instrs):
    """
    >>> execute_until_loop(parse_instructions('jmp +0'))
    (<State.LOOPED>, 0)
    >>> execute_until_loop(parse_instructions('''
    ...     nop +0  
    ...     acc +1
    ...     jmp +4
    ...     acc +3
    ...     jmp -3
    ...     acc -99
    ...     acc +1
    ...     jmp -4
    ...     acc +6
    ... '''))
    (<State.LOOPED>, 5)
    >>> execute_until_loop(parse_instructions('''
    ...     nop +0
    ...     acc +1
    ...     jmp +4
    ...     acc +3
    ...     jmp -3
    ...     acc -99
    ...     acc +1
    ...     nop -4
    ...     acc +6
    ... '''))
    (<State.TERMINATED>, 8)
    """
    acc = ip = 0
    state = State.RUNNING
    exec_counts = [0] * len(instrs)
    while True:
        if ip >= len(instrs):
            return state.TERMINATED, acc
        if exec_counts[ip]:
            return state.LOOPED, acc

        op, arg = instrs[ip]
        exec_counts[ip] += 1    
        if op == 'nop':
            ip += 1
        elif op == 'jmp':
            ip += arg
        elif op == 'acc':
            acc += arg
            ip += 1
        else:
            raise Exception(f'Unknown op: {op}')

def suggest_fixed_instructions(instrs):
    """
    >>> list(suggest_fixed_instructions([('jmp', 0), ('acc', 3), ('nop', 1)]))
    [[('nop', 0), ('acc', 3), ('nop', 1)], [('jmp', 0), ('acc', 3), ('jmp', 1)]]
    """
    for i in range(len(instrs)):
        op, arg = instrs[i]
        if op == 'jmp':
            op = 'nop'
        elif op == 'nop':
            op = 'jmp'
        else:
            continue
        yield instrs[:i] + [(op, arg)] + instrs[i+1:]

def fix_and_execute_until_termination(instrs):
    """
    >>> fix_and_execute_until_termination(parse_instructions('''
    ...     nop +0  
    ...     acc +1
    ...     jmp +4
    ...     acc +3
    ...     jmp -3
    ...     acc -99
    ...     acc +1
    ...     jmp -4
    ...     acc +6
    ... '''))
    8
    """
    for maybe_fixed_instrs in suggest_fixed_instructions(instrs):
        state, acc = execute_until_loop(maybe_fixed_instrs)
        if state == State.TERMINATED:
            return acc

In [54]:
doctest.run_docstring_examples(parse_instruction, globs=None, verbose=True)
doctest.run_docstring_examples(execute_until_loop, globs=None, verbose=True)
doctest.run_docstring_examples(suggest_fixed_instructions, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_instruction('acc -99')
Expecting:
    ('acc', -99)
ok
Trying:
    parse_instruction('nop +0')
Expecting:
    ('nop', 0)
ok
Finding tests in NoName
Trying:
    execute_until_loop(parse_instructions('jmp +0'))
Expecting:
    (<State.LOOPED>, 0)
ok
Trying:
    execute_until_loop(parse_instructions('''
        nop +0  
        acc +1
        jmp +4
        acc +3
        jmp -3
        acc -99
        acc +1
        jmp -4
        acc +6
    '''))
Expecting:
    (<State.LOOPED>, 5)
ok
Trying:
    execute_until_loop(parse_instructions('''
        nop +0
        acc +1
        jmp +4
        acc +3
        jmp -3
        acc -99
        acc +1
        nop -4
        acc +6
    '''))
Expecting:
    (<State.TERMINATED>, 8)
ok
Finding tests in NoName
Trying:
    list(suggest_fixed_instructions([('jmp', 0), ('acc', 3), ('nop', 1)]))
Expecting:
    [[('nop', 0), ('acc', 3), ('nop', 1)], [('jmp', 0), ('acc', 3), ('jmp', 1)]]
ok


In [56]:
# Final answers
with open('day8.txt') as f:
    instrs = parse_instructions(f.read())
    print('Part 1: ', execute_until_loop(instrs)[1])
    print('Part 2: ', fix_and_execute_until_termination(instrs))

Part 1:  1420
Part 2:  1245


# Day 9: Encoding Error

In [100]:
from itertools import combinations, islice

def first_invalid_number(ns, prev_n=25):
    """
    >>> first_invalid_number([
    ...     35, 20, 15, 25, 47, 40, 62, 55, 65,
    ...     95, 102, 117, 150, 182, 127, 219
    ... ], prev_n=5)
    127
    """
    for i, n in islice(enumerate(ns), prev_n, None):
        preamble = ns[i-prev_n:i]
        if any(sum(p) == ns[i] for p in combinations(preamble, 2)):
            continue
        return n


def span_summing_to(ns, n):
    """
    >>> span_summing_to([35, 20, 15, 25, 47, 40, 62, 55, 65], 127)
    [15, 25, 47, 40]
    """
    for i in range(len(ns)):
        total = 0
        for j in range(i, len(ns)):
            total += ns[j]
            if total == n:
                return ns[i:j+1]
            elif total > n:
                break
            

def encryption_weakness(ns, prev_n=25):
    """
    >>> encryption_weakness([
    ...     35, 20, 15, 25, 47, 40, 62, 55, 65,
    ...     95, 102, 117, 150, 182, 127, 219
    ... ], prev_n=5)
    62
    """
    invalid_num = first_invalid_number(ns, prev_n)
    rs = span_summing_to(ns, invalid_num)
    return min(rs) + max(rs)

In [101]:
doctest.run_docstring_examples(first_invalid_number, globs=None, verbose=True)
doctest.run_docstring_examples(span_summing_to, globs=None, verbose=True)
doctest.run_docstring_examples(encryption_weakness, globs=None, verbose=True)

Finding tests in NoName
Trying:
    first_invalid_number([
        35, 20, 15, 25, 47, 40, 62, 55, 65,
        95, 102, 117, 150, 182, 127, 219
    ], prev_n=5)
Expecting:
    127
ok
Finding tests in NoName
Trying:
    span_summing_to([35, 20, 15, 25, 47, 40, 62, 55, 65], 127)
Expecting:
    [15, 25, 47, 40]
ok
Finding tests in NoName
Trying:
    encryption_weakness([
        35, 20, 15, 25, 47, 40, 62, 55, 65,
        95, 102, 117, 150, 182, 127, 219
    ], prev_n=5)
Expecting:
    62
ok


In [102]:
# Final answers
with open('day9.txt') as f:
    ns = [int(l) for l in f]
    print('Part 1: ', first_invalid_number(ns))
    print('Part 2: ', encryption_weakness(ns))

Part 1:  21806024
Part 2:  2986195
